<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Introduction to Support Vector Machines (SVM)


---

### Learning Objectives
- Understand how the SVM builds its decision threshold
- Understand the concept of the maximum margin hyperplane
- Visualize the linearly separable case in classification
- Understand the math behind finding the maximum margin hyperplane
- Understand the hinge loss for SVM
- Understand how the regularization constant C allows SVMs to fit non-linearly separable problems
- See how the kernel trick transforms problems from non-linearly separable to linearly separable

### Lesson Guide
- [Introduction to SVMs](#intro)
- [How does the SVM classifiy?](#how-classify)
- [Intuition behind the decision boundary](#intuition)
- [Why maximize the margin?](#why-max-margin)
- [SVM origins: the perceptron algorithm](#perceptron)
- [The maximum margin hyperplane](#mmh)
    - [Finding the maximum margin](#finding-mmh)
- [The hinge loss](#hinge)
    - ["Slack"](#slack)
    - [The regularizing hyperparameter C](#hyper-c)
- [The kernel trick](#kernel)
- [Additional resources](#resources)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from ipywidgets import *
from IPython.display import display

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

<a id='intro'></a>

## Introduction to SVMs

---

The Support Vector Machine (SVM) algorithm is a different approach to classification.

SVM still fits a decision boundary like a logistic regression, but uses a different loss function called the "hinge loss" (as opposed to the log loss in logistic regression).

This lesson will overview the details of the SVM algorithm.

<a id='how-classify'></a>

## How does the SVM classify?

---

It's important to start with the intuition for SVM with the special _**linearly separable**_ classification case.

If classification of observations is "linearly separable", SVM fits the **"decision boundary"** that is defined by the largest margin between the closest points for each class. This is commonly called the **"maximum margin hyperplane (MMH)"**.

![linearly separable SVM](./assets/linear_separability_vs_not.png)

<a id='intuition'></a>

## Intuition behind the SVM decision boundary

---

SVM's criterion for a decision surface is one that is _maximally far away from any data point between classes_. The distance from the decision boundary to the closest data point determines the "margin" of the classifier.

The points SVM uses to fit the decision boundary are called "support vectors".

![linearly separable SVM MMH margin](./assets/Margin.png)

<a id='why-max-margin'></a>

## Why maximize the margin?

---

**SVM solves for a decision boundary that should theoretically minimize the generalization error.** 

Observations that are near the decision boundary between the classes are the observations with the most "ambiguous" labels. They are the observations that are approaching equal probability to be one class or the other (given the predictors).

SVM, instead of considering all the observations "equally" in the loss function it minimizes, defines it's fit using the most ambiguous points. Its decision boundary is safe in that errors in new measured observations are not likely to cause the SVM to mis-classify.

The SVM is concerned with generalization to new data.

<a id='perceptron'></a>

## SVM origins: the perceptron algorithm

---

The perceptron algorithm is a linear function of weights $w$ and predictors $X$ that assigns points to binary classes. If the function returns a value greater than 0 then the observation is classified as 1, otherwise it is classified as zero.

$f(X)$ below is the perceptron function to determine the classes. $b$ here is known as the "bias", which is analagous to the intercept term.

$$ f(X) = \begin{cases}1 & \text{if }w^T x + b > 0\\0 & \text{otherwise}\end{cases} $$

If the points are linearly seperable, then solving the perceptron is guaranteed to converge on a solution, but that solution is not necessarily optimal for future observations. This led to the invention of the SVM, which finds the optimal discriminator: the maximum margin hyperplane.



<a id='mmh'></a>

## The maximum margin hyperplane

---

We choose a normalizing constant such that the distance from the plane defined by

$$ w^T x + b = 0$$

to the closest points (known as "support vectors") of either classes will be 1:

$$ w^T x_{+} + b = 1 \\ w^T x_{-} + b = -1 $$

If the norm for the weights is $||w||$, the size of the margin is

$$ \text{margin} = \frac{2}{||w||} $$


![linearly separable SVM MMH margin](./assets/linear_sep_support_vecs_math.png)

<a id='finding-mmh'></a>
### Finding the maximum margin

We want to find a distance between these points that is the widest possible. Therefore, we are looking to maximize the value $\frac{1}{||w||}$.

So, maximize the weights with constraints on what the function can output. When the target is 1, the function needs to be 1 or greater. When the target is 0 (or -1), the function needs to output -1 or lower.

$$ \underset{w}{\text{max}} \frac{1}{||w||} \quad \text{subject to} \begin{cases}\text{if } y_i = 1 & w^T x_i + b \ge 1 \\ \text{if } y_i = -1 & w^T x_i + b \le -1 \end{cases} \text{for } i \text{ in } N$$

Note here that $y$ is specified as either 1 or -1, as opposed to the 0, 1 that we are used to. This is convenient for formulating the optimization problem in a concise way. 

Additionally, we can change the optimization problem into a minimization problem by requiring

$$ \underset{w}{\text{min }} \frac{||w||^2}{2} \quad \text{subject to} \quad y_i(w^T x_i + b) \ge 1 $$

which works out to be the same as above with $y_i=1$ or $y_i = -1$.

![linearly separable SVM MMH margin](./assets/linear_sep_support_vecs.png)

<a id='hinge'></a>

## The hinge loss and non-linearly separable cases

---

In cases where there is no line or plane that can separate all of the points perfectly, we need to introduce the capacity for model error. Using the constraint above that $y_i(w^T x_i + b) \ge 1$, we can introduce the **hinge loss function**:

$$ 
\text{hinge loss} = \sum_{i=1}^n \max\left(0, 1 - y_i(w^T x_i + b)\right) $$

where now, if the point is on the correct side (correctly classified), the value will be 0. If the point is not $\ge 1$, this function will be greater than zero.

Using $f(x_i) = (w^T x_i + b)$, 

$$\text{hinge loss} = \sum_{i=1}^N {\rm max}\left(0, 1 - y_i \: f(x_i)\right)$$ 

If $y_i\, f(x_i) > 1$, the point lies _outside_ the margin and does not contribute to the loss.

If $y_i\, f(x_i) = 1$ the point is _on_ the margin and does not contribute to the loss.

If $y_i\, f(x_i) < 1$ the point lies _inside_ the margin and **does** contribute to the loss.


![Hinge loss](./assets/hinge_loss.png)

<a id='slack'></a>

### Hinge loss and "slack"

When we have a scenario where it is not possible to perfectly separate, we allow some of the points to be on the wrong side by an amount $\xi_i\ge 0$ (we do not measure how much points are on the correct side). 
We will allow only for a certain total budget of being on the wrong side:

$$
\sum_{i=1}^N\xi_i\le {\rm constant}
$$

We control this budget using a regularization constant $C$. Large values of $C$ strongly penalize being on the wrong side, low values of $C$ hardly penalize being on the wrong side. The $\xi_i$ are called **slack variables**.

The modified hinge loss with a regularization constant $C$ thus becomes

$$ \min \frac{||w||^2}{2} + C\sum_{i=1}^N \xi_i \quad \text{subject to} \quad y_i(w^T x_i + b) \ge 1 - \xi_i \quad \text{and} \quad \xi_i\ge 0$$

where the $\xi_i$ are the errors of the classifier, and $C$ is a regularization constant that determines how much the classification errors matter (relative to the maximization of the margin).

The function that the SVM minimizes to find the boundary is:

$$  \underset{w}{\text{min }} \frac{||w||^2}{2} + C\sum_{i=1}^N {\rm max}\left(0, 1 - y_i(w^T x_i + b)\right) $$

A small $C$ creates a wider margin because errors will matter less. A large $C$ creates a tighter margin because errors matter more. An infinite $C$ parameter is a "hard" margin, which always minimizes error over the size of the boundary.

We are trying to minimize the norm of the weights $||w||$ and thus maximize the margin, but now we are also trying to minimize our errors at the same time. We have a balance in our minimization between how wide the margin should be and how much error we tolerate.

<a id='hyper-c'></a>
### The regularizing hyper-parameter $C$

By setting the hyper-parameter $C$ we can control the extent to which our SVM is tolerant to misclassification errors. It is sometimes called the "soft-margin constant". 

$C$ affects the strength of the "penalty", similar to the $C$ terms in the Ridge and Lasso. 

By multiplying the sum of the errors, which are the distances from the margins to the points inside the margin, it allows the SVM to classify non-linearly separable problems by allowing errors to occur. 

The lower the value of $C$, the more misclassified observation errors are allowed. These errors are known as "slack variables". Reducing the effect of errors on the loss function puts the emphasis on widening the margin.

For those interested in exploring the math more, [there is a good tutorial here.](http://www.svm-tutorial.com/2015/06/svm-understanding-math-part-3/#more-457)

![soft margin](./assets/slack_variables.png)

![soft margin](./assets/soft_margin.png)

<a id='kernel'></a>
## The "kernel trick" for non-linearly separable problems

---

The "kernel trick" allows an SVM to classify non-linearly separable problems. It is a big reason why SVMs are so popular.

The idea behind the kernel trick is that you can arbitrarily transform your observations that _have no linear separability_ by putting them into a different "dimensional space" where they DO have linear separability, fit an SVM in that higher dimensional space, and then invert the transformation of the data and the model itself back into the original space.

This is done by "wrapping" your predictors in a kernel function that transforms them into this higher dimensional space. 

[Check out these lecture slides for more detail.](http://www.robots.ox.ac.uk/~az/lectures/ml/lect3.pdf)

The following pictures should give you a general intuition for what is happening.

![kernel transform viz](./assets/kernel_trick.png)

![polynomial kernel](./assets/nonlinear-1.png)

![gaussian kernel](./assets/nonlinear-2.png)

## Kernel functions

dth-degree polynomial:

$$
K(x,x') = \left(r+\gamma\langle x,x'\rangle\right)^d
$$

radial basis:

$$
K(x,x') = \exp\left(-\gamma \|x-x'\|^2\right)
$$


sigmoid:

$$
K(x,x') = \tanh\left(\gamma\langle x,x'\rangle + r\right)
$$

Here

$$
\langle x,x'\rangle = \sum_{i=1}^p x_i x'_i
$$

stands for the scalar product,
$d$, $\gamma$, $r$ are tuning constants.

Sklearn chooses the following names for these parameters in its SVM-classifier:

- $d$ $\rightarrow$ degree (default 3)
- $\gamma$ $\rightarrow$ gamma (default 1)
- $r$ $\rightarrow$ coef0 (default 0)

The parameters only have an effect if they appear in the kernel definition.

A prediction for any point $x$ is made by calculating the kernel for this point paired up with all the support vectors:

$$
\hat{f}(x) = \sum_{i=1}^N \hat\alpha_i y_i K(x,x_i) + \hat{\beta}_0
$$

The coefficients $\hat{\alpha}_i$ are the so-called dual coefficients. They are only different from zero if $x_i$ is a support vector, so the sum effectively runs only over the support vectors.

SVM docs on [SKLearn](http://scikit-learn.org/stable/modules/svm.html)

<a id='resources'></a>

## Additional resources

---

- [For a really great resource check out these slides (some of which are cannabalized in this lecture).](http://www.robots.ox.ac.uk/~az/lectures/ml/lect2.pdf)
- [This website is also a great resource, on a slightly more technical level.](http://nlp.stanford.edu/IR-book/html/htmledition/support-vector-machines-the-linearly-separable-case-1.html)
- SVM docs on [SKLearn](http://scikit-learn.org/stable/modules/svm.html)
- Iris example on [SKLearn](http://scikit-learn.org/stable/auto_examples/svm/plot_iris.html#example-svm-plot-iris-py)
- Hyperplane walkthrough on [SKLearn](http://scikit-learn.org/stable/auto_examples/svm/plot_separating_hyperplane.html#example-svm-plot-separating-hyperplane-py)
- A comprehensive [user guide](http://pyml.sourceforge.net/doc/howto.pdf) to SVM. My fav!
- A [blog post tutorial](http://www.svm-tutorial.com/2014/11/svm-understanding-math-part-2/) of understanding the linear algebra behind SVM hyperplanes. Check [part 3](http://www.svm-tutorial.com/2015/06/svm-understanding-math-part-3/) of this blog on finding the optimal hyperplane
- This [Quora discussion](https://www.quora.com/How-do-you-teach-Support-Vector-Machine-to-a-beginner-in-Machine-Learning) includes a high-level overview plus a [20min video](https://www.youtube.com/watch?v=aDbsJ_S3tIA) walking through the core "need-to-knows"
- A [slideshow introduction](http://www.cs.columbia.edu/~kathy/cs4701/documents/jason_svm_tutorial.pdf) to the optimization considerations of SVM
- A second [slideshow overview from UCF](http://www.cs.ucf.edu/courses/cap6412/fall2009/papers/Berwick2003.pdf) on the highnotes of SVM
- Andrew Ng's [notes](http://cs229.stanford.edu/notes/cs229-notes3.pdf) on SVM from CS 229
- A [FULL LECTURE](https://www.youtube.com/watch?v=eHsErlPJWUU) (1hr+) from one of my fav lecturers (Dr Yasser) on SVM. He does a followup on [kernel tricks](https://www.youtube.com/watch?v=XUj5JbQihlU) too
- A [FULL LECTURE](https://www.youtube.com/watch?v=_PwhiWxHK8o) (50min) (from MIT Opencoursewar)
- An infamous [paper](https://www.cs.cornell.edu/people/tj/publications/joachims_98a.pdf) (cited 7000+ times!) on why SVM is a great text classifier
- An [advanced discussion](http://www.icml-2011.org/papers/386_icmlpaper.pdf) of SVMs as probabilistic models